<a href="https://colab.research.google.com/github/mallickhiba/Image-Denoising-with-Parallel-Distributed-Non-Local-Means/blob/main/Image_Denoising_with_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Set Up

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-rvf98t3l
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-rvf98t3l
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-ra06wszh/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpgamlugrw".


In [4]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

In [5]:
%%cuda
#include <stdio.h>
#include <iostream>

int main()
{
    std::cout << "Welcome to CUDA\n";
    return 0;
}

Welcome to CUDA



# Project

In [6]:
!git clone https://github.com/mallickhiba/Image-Denoising-with-Parallel-Distributed-Non-Local-Means.git

Cloning into 'Image-Denoising-with-Parallel-Distributed-Non-Local-Means'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 172 (delta 67), reused 171 (delta 66), pack-reused 0
Receiving objects: 100% (172/172), 3.43 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [7]:
# prompt: how do i move files from a folder in google colab into another
!cp -r /content/Image-Denoising-with-Parallel-Distributed-Non-Local-Means/NLM_CUDA/* /content/

In [8]:
mkdir -p data/out

In [9]:
mkdir build

In [18]:
!nvcc -o build/main -I./include /content/src/main.cu -O3

In [19]:
#./build/main $(imageNum) $(patchSize) $(filterSigma) $(patchSigma) $(useGpu) $(useSharedMem)
#seq
!./build/main 0 7 0.06 0.8 !useGpu !useSharedMem


Image: House

Image read

CPU filtering time: 3916.15ms

Image filtered: 
-Patch size 7
-Patch sigma 0.8
-Filter Sigma 0.06

Residual calculated

Filtered image written

Residual written




In [12]:
#./build/main $(imageNum) $(patchSize) $(filterSigma) $(patchSigma) $(useGpu) $(useSharedMem)
!./build/main 1 7 0.06 0.8 useGpu !useSharedMem


Image: Flower

Image read
^C


In [13]:
#./build/main $(imageNum) $(patchSize) $(filterSigma) $(patchSigma) $(useGpu) $(useSharedMem)
!./build/main 2 7 0.06 0.8 useGpu useSharedMem


Image: Lena

Image read
^C


## Showing output

In [23]:
import numpy as np
import matplotlib.pyplot as plt

# Read the grayscale pixel values from the text file
# Open the file for reading
with open('/content/residual.txt', 'r') as f:
    # Read the file line by line
    pixel_values = []
    for line in f:
        # Split the line into a list of values using commas
        values = line.split(' ')
        # Convert each value to a float and add it to the list
        for value in values:
            pixel_values.append(float(value))

# Create a 2D array from the pixel values
image_array = np.array(pixel_values).reshape((64, 64))

# Display the image using matplotlib
plt.imshow(image_array, cmap='gray')
plt.axis('off')
plt.show()


ValueError: could not convert string to float: '\n'

In [ ]:
# prompt: given th path to a txt file with greyscale pxiel values for an image like this 0.430538 0.384078 . use matplotlib to visualise and display the image

import numpy as np
import matplotlib.pyplot as plt

# Read the grayscale pixel values from the text file
# Open the file for reading
with open('/content/data/out/f', 'r') as f:
  # Read the file line by line
  pixel_values = []
  for line in f:
    # Split the line into a list of values
    values = line.split()
    # Convert each value to a float and add it to the list
    for value in values:
      pixel_values.append(float(value))

# Create a 2D array from the pixel values
image_array = np.array(pixel_values).reshape((64, 64))

# Display the image using matplotlib
plt.imshow(image_array, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# prompt: given th path to a txt file with greyscale pxiel values for an image like this 0.430538 0.384078 . use matplotlib to visualise and display the image

import numpy as np
import matplotlib.pyplot as plt

# Read the grayscale pixel values from the text file
# Open the file for reading
with open('/content/data/out/residual_5_0.060000_0.800000.txt', 'r') as f:
  # Read the file line by line
  pixel_values = []
  for line in f:
    # Split the line into a list of values
    values = line.split()
    # Convert each value to a float and add it to the list
    for value in values:
      pixel_values.append(float(value))

# Create a 2D array from the pixel values
image_array = np.array(pixel_values).reshape((64, 64))

# Display the image using matplotlib
plt.imshow(image_array, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
import cv2
import numpy as np

def add_gaussian_noise(image, mean=0, std_dev=25):
    """
    Add Gaussian noise to the image.

    Parameters:
        image (numpy.ndarray): Input image.
        mean (float): Mean of the Gaussian distribution.
        std_dev (float): Standard deviation of the Gaussian distribution.

    Returns:
        numpy.ndarray: Noisy image.
    """
    noise = np.random.normal(mean, std_dev, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

def add_salt_and_pepper_noise(image, amount=0.02):
    """
    Add salt-and-pepper noise to the image.

    Parameters:
        image (numpy.ndarray): Input image.
        amount (float): Probability of the noise.

    Returns:
        numpy.ndarray: Noisy image.
    """
    noisy_image = np.copy(image)
    num_salt = np.ceil(amount * image.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_salt))
          for i in image.shape]
    noisy_image[coords[0], coords[1], :] = 255

    num_pepper = np.ceil(amount* image.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_pepper))
          for i in image.shape]
    noisy_image[coords[0], coords[1], :] = 0
    return noisy_image

# Load an image
image_path = 'input_image.jpg'
image = cv2.imread(image_path)

# Add Gaussian noise
noisy_image_gaussian = add_gaussian_noise(image)

# Add salt-and-pepper noise
noisy_image_salt_pepper = add_salt_and_pepper_noise(image)

# Display the original and noisy images
cv2.imshow('Original Image', image)
cv2.imshow('Noisy Image (Gaussian)', noisy_image_gaussian)
cv2.imshow('Noisy Image (Salt-and-Pepper)', noisy_image_salt_pepper)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt

# Function to add noise to an image
def add_noise(image, noise_level=0.1):
    """
    Add random noise to the image.

    Parameters:
    image (PIL.Image): The input image.
    noise_level (float): Noise level between 0 and 1.

    Returns:
    PIL.Image: Image with added noise.
    """
    np.random.seed(42)  # Set seed for reproducibility
    img_array = np.array(image)  # Convert image to numpy array

    # Generate random noise
    noise = np.random.normal(loc=0, scale=255*noise_level, size=img_array.shape).astype(np.uint8)

    # Add noise to the image
    noisy_image = np.clip(img_array + noise, 0, 255).astype(np.uint8)

    return Image.fromarray(noisy_image)

# Load an image (replace 'image_path.jpg' with your image path)
image_path = "/content/Screenshot 2024-04-14 015739.png"  # Modify this path
image = Image.open(image_path)

# Display original image
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")
plt.axis("off")

# Add noise to the image
noisy_image = add_noise(image)

# Display image with added noise
plt.subplot(1, 2, 2)
plt.imshow(noisy_image)
plt.title("Image with Added Noise")
plt.axis("off")

plt.show()


In [ ]:
# prompt: write code to take an image from content and resize it and save it in content with a new name

from PIL import Image

# Open the image from content
image = Image.open('/content/image.jpg')

# Resize the image
resized_image = image.resize((224, 224))

# Save the resized image
resized_image.save('/content/resized_image.jpg')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Read the grayscale pixel values from the text file
# Open the file for reading
with open('/content/data/images/input.jpg', 'r') as f:
    # Read the file line by line
    pixel_values = []
    for line in f:
        # Split the line into a list of values using commas
        values = line.split(',')
        # Convert each value to a float and add it to the list
        for value in values:
            pixel_values.append(float(value))

# Create a 2D array from the pixel values
image_array = np.array(pixel_values).reshape((64, 64))

# Display the image using matplotlib
plt.imshow(image_array, cmap='gray')
plt.axis('off')
plt.show()


In [ ]:
# prompt: USE AN JPG IMAGE FROM CONTENT AND TURN IT INTO A NP.ARRAY OF PIXEL VALUES AND SAVE IT AS A TXT FILE TO A FOLDER CALLED IN IN THE CONTENT.

# Import necessary libraries
import numpy as np
from PIL import Image

# Open the image from content
image = Image.open('/content/data/images/input3.jpg')

# Convert the image to grayscale
image = image.convert('L')

# Resize the image
resized_image = image.resize((256, 256))

# Convert the image to a numpy array
image_array = np.array(resized_image)

# Save the image array to a text file
with open('/content/data/in/input3.txt', 'w') as f:
    for row in image_array:
        for pixel in row:
            f.write(str(pixel) + ',')
        f.write('\n')
